In [1]:
from google.colab import drive
drive.mount("./drive")

Mounted at ./drive


# Downloading Dataset and saving it inside Drive

In [ ]:
# !wget http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/Avenue_Dataset.zip
# !wget http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/ground_truth_demo.zip
# !mkdir ./Avenue_Dataset
# !unzip /content/Avenue_Dataset.zip -d ./Avenue_Dataset
# !unzip ./ground_truth_demo.zip -d ./Avenue_Dataset
# !wget http://www.svcl.ucsd.edu/projects/anomaly/UCSD_Anomaly_Dataset.tar.gz
# !mkdir ./UCSD_Anomaly_Dataset
# !tar -xzvf ./UCSD_Anomaly_Dataset.tar.gz -C ./UCSD_Anomaly_Dataset
# !mkdir ./drive/MyDrive/AnormalyDetectionDataset
# !mv ./Avenue_Dataset ./drive/MyDrive/AnormalyDetectionDataset
# !mv ./UCSD_Anomaly_Dataset ./drive/MyDrive/AnormalyDetectionDataset

--2021-03-05 08:39:34--  http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/Avenue_Dataset.zip
Resolving www.cse.cuhk.edu.hk (www.cse.cuhk.edu.hk)... 137.189.91.192
Connecting to www.cse.cuhk.edu.hk (www.cse.cuhk.edu.hk)|137.189.91.192|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813227845 (776M) [application/zip]
Saving to: ‘Avenue_Dataset.zip’

Avenue_Dataset.zip  100%[===================>] 775.55M  4.55MB/s    in 2m 44s  

2021-03-05 08:42:18 (4.74 MB/s) - ‘Avenue_Dataset.zip’ saved [813227845/813227845]



# Importing Data from Drive

In [2]:
data_path = 'drive/myDrive/AnormalyDetectionDataset'
!ls ./drive/MyDrive/AnormalyDetectionDataset/UCSD_Anomaly_Dataset

UCSD_Anomaly_Dataset.v1p2


# Importing Modules

In [3]:
!pip install livelossplot

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *

from tensorflow.keras.utils import plot_model
from livelossplot import PlotLossesKeras
from tensorflow.keras.callbacks import *
from keras import backend as K

from imutils import build_montages
import os
from PIL import Image
import cv2
from collections import Counter
import imutils
from imutils import paths

from scipy.spatial.distance import cosine, euclidean
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly as ply

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
ply.offline.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Creating the Dataset

In [11]:
train_path = "./drive/MyDrive/AnormalyDetectionDataset/Avenue_Dataset/Avenue Dataset/training_videos"
test_path = "./drive/MyDrive/AnormalyDetectionDataset/Avenue_Dataset/Avenue Dataset/testing_videos"

In [ ]:
store_imgs = []
fps = 5
train_videos = os.listdir(train_path)
train_images_path = train_path + '/frames'
os.makedirs(train_images_path)

# Utility Functions

In [26]:
# To collect frames from videos and make a dataset out of it
def store_in_array(image_path):
    image = load_img(image_path)
    image = img_to_array(image)
    image = cv2.resize(
        image, 
        (227, 227),
        interpolation = cv2.INTER_AREA
        )
    gray = 0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_imgs.append(gray)

In [27]:
train_videos

['01.avi',
 '02.avi',
 '03.avi',
 '04.avi',
 '05.avi',
 '06.avi',
 '07.avi',
 '08.avi',
 '09.avi',
 '10.avi',
 '11.avi',
 '12.avi',
 '14.avi',
 '13.avi',
 '15.avi',
 '16.avi',
 'frames']

In [29]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [38]:
for video in tqdm(train_videos):
    os.system('ffmpeg -i "{}/{}" -r 1/{}  "{}/frames/%03d.jpg"'.format(train_path,video,fps,train_path))
    images=os.listdir(train_images_path)
    for image in images:
        image_path=train_images_path + '/' + image
        store_in_array(image_path)

100%|██████████| 17/17 [00:30<00:00,  1.77s/it]


In [39]:
store_imgs = np.array(store_imgs)
a, b, c = store_imgs.shape

store_imgs.resize(b, c, a)
store_imgs = (store_imgs-store_imgs.mean())/(store_imgs.std())
store_imgs = np.clip(store_imgs, 0, 1)
np.save(f'./drive/MyDrive/AnormalyDetectionDataset/training.npy', store_imgs)

# Building Spatio-Temporal AutoEncoder

In [67]:
class ST_Autoencoder:
        
    def __init__(self, input_dim, z_dim):         
        self.input_dim = input_dim
        self.z_dim = z_dim
        print("[INFO] Build your model by calling build()..")

    def build(self):
        # Encoder
        encoder_input = Input(shape=self.input_dim, name='encoder_input')

        enc_x = encoder_input
        enc_x = Conv3D(filters = 128,
                   kernel_size = (11, 11, 1),
                   strides = (4, 4, 1),
                   padding = 'valid',
                   kernel_initializer = 'he_uniform',
                   activation = 'relu'
                   )(enc_x)
        enc_x = Conv3D(filters = 64,
                   kernel_size = (5, 5, 1),
                   strides = (2, 2, 1),
                   padding = 'valid',
                   kernel_initializer = 'he_uniform',
                   activation = 'relu'
                   )(enc_x)
        enc_x = ConvLSTM2D(filters=64,
                       kernel_size=(3,3),
                       strides=1,
                       padding='same',
                       dropout=0.4,
                       recurrent_dropout=0.3,
                       return_sequences=True
                    )(enc_x)
        embedding = ConvLSTM2D(filters=self.z_dim,
                       kernel_size=(3,3),
                       strides=1,
                       padding='same',
                       dropout=0.3,                       
                       return_sequences=True
                    )(enc_x)

        # Decoder
        dec_x = ConvLSTM2D(filters=64,
                           kernel_size=(3,3),
                           strides=1,
                           return_sequences=True, 
                           padding='same',
                           dropout=0.5
                        )(embedding)
        dec_x = Conv3DTranspose(filters=128,
                                kernel_size=(5,5,1),
                                strides=(2,2,1),
                                padding='valid',
                                activation='relu'
                            )(dec_x)
        decoder_output = Conv3DTranspose(filters=1,
                                kernel_size=(11,11,1),
                                strides=(4, 4, 1),
                                padding='valid',
                                activation='relu'
                            )(dec_x)       



        stae = Model(inputs=encoder_input, outputs=decoder_output, name="Spatio-Temporal-AutoEncoder")
        stae.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
        print()
        print("=======================================")
        print("[INFO] Model compilation Done..")
        print("=======================================")
        print()
        print(stae.summary())
        return stae

    
    def __str__():
        return "Spatio-Temporal-AutoEncoder"

In [68]:
modelBuilder = ST_Autoencoder(input_dim=(227, 227, 10, 1),
                              z_dim = 32)

[INFO] Build your model by calling build()..


In [69]:
stae = modelBuilder.build()


[INFO] Model compilation Done..

Model: "Spatio-Temporal-AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 227, 227, 10, 1)] 0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 55, 55, 10, 128)   15616     
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 26, 26, 10, 64)    204864    
_________________________________________________________________
conv_lst_m2d_18 (ConvLSTM2D) (None, 26, 26, 10, 64)    295168    
_________________________________________________________________
conv_lst_m2d_19 (ConvLSTM2D) (None, 26, 26, 10, 32)    110720    
_________________________________________________________________
conv_lst_m2d_20 (ConvLSTM2D) (None, 26, 26, 10, 64)    221440    
_________________________________________________________________
conv3

# Loading our Image Data

In [59]:
train_data = np.load('./drive/MyDrive/AnormalyDetectionDataset/training.npy')
frames = train_data.shape[2]
frames = frames - frames%10

train_data = train_data[:, :, :frames]
train_data = train_data.reshape(-1, 227, 227, 10)

train_data = np.expand_dims(train_data, axis = 4)
target_data = train_data.copy()

# Training

In [70]:
epochs = 5
batch_size = 1

if not os.path.exists("./drive/MyDrive/AnormalyDetectionDataset/Model"):
    os.mkdir("./drive/MyDrive/AnormalyDetectionDataset/Model")

model_save = ModelCheckpoint("./drive/MyDrive/AnormalyDetectionDataset/Model/stae.h5",
                            monitor = 'mean_squared_error',
                            save_best_only = True)
early_stop = EarlyStopping(monitor = 'mean_squared_error',
                        patience = 3,
                        mode = 'min',
                        restore_best_weights = True)

callbacks = [early_stop, model_save]

In [71]:
training_history = stae.fit(x = train_data, 
                            y = target_data,
                            batch_size = batch_size,
                            epochs = epochs,
                            callbacks = callbacks)

Epoch 1/5
23/23 [==============================] - 45s 301ms/step - loss: 0.2696 - accuracy: 0.5155
Epoch 2/5
23/23 [==============================] - 7s 289ms/step - loss: 0.2068 - accuracy: 0.5450
Epoch 3/5
23/23 [==============================] - 7s 303ms/step - loss: 0.2005 - accuracy: 0.5437
Epoch 4/5
23/23 [==============================] - 7s 292ms/step - loss: 0.2011 - accuracy: 0.5428
Epoch 5/5
23/23 [==============================] - 7s 288ms/step - loss: 0.1971 - accuracy: 0.5472


# Saving STAE

In [74]:
stae.save("./drive/MyDrive/AnormalyDetectionDataset/Model/stae.h5")